In [ ]:
import requests
import json
import math
import os
import csv
from collections import Counter
from enum import Enum

In [ ]:
username = 'login'
senha = 'psw'

In [ ]:
class Operations(Enum):
    Login = 'login'
    Logout = 'logout'
    DatasetFields = 'datasetfields'
    Search = 'search'
    DownloadOptions = 'downloadoptions'
    Download = 'download'
    IdLookUp = 'idlookup'
    MetaData = 'metadata'
    UpdateBulk = 'updatebulkdownloadscene'
    SubmitBulk = 'submitbulkdownloadorder'

In [ ]:
'a' + Operations.Login.value

In [ ]:
defaultUrl = 'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/'
legacyUrl = 'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/'
operations = {'login': 'login',
              'logout' : 'logout',
              'datasetfields' : 'datasetfields',
              'search' : 'search',
              'downloadoptions' : 'downloadoptions',
              'download' : 'download',
              'idlookup' : 'idlookup',
              'metadata' : 'metadata',
              'updateBulk' : 'updatebulkdownloadscene',
              'submitBulk' : 'submitbulkdownloadorder'
             }

productTypes = {'full' : 'STANDARD',
                'preview' : 'FRB'
               }

downloadIndexes = {'displayId' : 0,
                   'downloadUrl' : 1,
                   'entityId' : 2,
                   'startTime' : 3,
                   'endTime' : 4
                  }

In [ ]:
def login(usr,pwd, catalog = 'EE'):
    payload = {"jsonRequest" : '{"username":"' + usr + '","password":"' + pwd + '","catalogId":"' + catalog + '"}'}
    response = requests.post(defaultUrl + operations['login'],data = payload)
    if response.json()['errorCode'] is None:
        return response.json()['data']
    else:
        print("Login inválido!")
        return None

In [ ]:
def logout(apiKey):
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey+ '"}'}
    return requests.post(defaultUrl + operations['logout'],data=payload).json()['data']

In [ ]:
def calculateMBR(point,radius):
    coef = radius * 0.0000089;
    ll = (point[0] - coef,point[1] - coef / math.cos(point[0] * 0.018))
    ur = (point[0] + coef,point[1] + coef / math.cos(point[0] * 0.018))
    return ll, ur

In [ ]:
def updateBulk(apiKey, downloadCodes, entityId, dataSetName = "SENTINEL_2A"):
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + dataSetName + '","downloadCodes":'+ downloadCodes +',"entityId":' + entityId + '}'}
    return requests.post(defaultUrl + operations['updateBulk'],data=payload).json()

In [ ]:
updateBulk('db000f703052452ab43057bf0b764807','["' +productTypes['full'] + '","' + productTypes['preview'] + '"]','1100852')

In [ ]:
updateBulk('db000f703052452ab43057bf0b764807','["' +productTypes['full'] + '","' + productTypes['preview'] + '"]','1464133')

In [ ]:
def submitBulk(apiKey):
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey+ '"}'}
    return requests.post(defaultUrl + operations['submitBulk'],data=payload).json()

In [ ]:
submitBulk("db000f703052452ab43057bf0b764807")

In [ ]:
def metadata(apiKey, entityIds, dataSetName = "SENTINEL_2A"):
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + dataSetName + '","entityIds":' + entityIds + '}'}
    return requests.post(defaultUrl + operations['metadata'],data=payload).json()

In [ ]:
a = metadata('d64e107c95c2457ea498cb20752f3d83','["1338121"]')
a

In [ ]:
print(a['data'][0]['spatialFootprint'])
print(a['data'][0]['sceneBounds'])
for metaDataField in a['data'][0]['metadataFields']:
    if metaDataField['fieldName'] == 'Cloud Cover':
        print(metaDataField['value'])
        break

In [ ]:
def sceneSearch(apiKey,point,radius,startDate,endDate,
                dataSetName = "SENTINEL_2A", months = None,
                includeUnknownCloudCover = True,maxResults = 1000, sortOrder = 'ASC'):
    ll, ur = calculateMBR(point,radius)
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + dataSetName + '","spatialFilter":{"filterType":"mbr","lowerLeft":{"latitude":"' + str(ll[0]) + '","longitude":"' + str(ll[1]) + '"},"upperRight":{"latitude":"' + str(ur[0]) + '","longitude":"' + str(ur[1]) + '"}},"temporalFilter":{"startDate":"' + startDate + '","endDate":"' + endDate + '"},'+( '' if months is None else '"months":' + str(months) + ',') + '"includeUnknownCloudCover":' + str(includeUnknownCloudCover).lower() + ',"maxResults":"' + str(maxResults) + '","sortOrder":"' + sortOrder + '"}'}
    response = requests.post(defaultUrl + operations['search'], data = payload)
    return response.json()

In [ ]:
def getScenesDict():
    if not os.path.isfile('scenes.json'):
        return json.loads('{}')
    else:
        with open('scenes.json','r') as jsonFile:
            return json.load(jsonFile)

In [ ]:
def downloadFile(downloadInfo,apiKey,datasetName = "SENTINEL_2A",productType = "STANDARD"):
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + datasetName + '","entityIds":"' + downloadInfo[downloadIndexes['entityId']] + '","products":"' + productType + '"}'}
    with requests.post(defaultUrl + operations['download'], data=payload) as download:
        downloadLink = (download.json())['data'][0]['url']
        with requests.get(downloadLink, stream=True, allow_redirects=True) as request:
            #lança exceção caso request de errado
            request.raise_for_status()
            filename = request.headers['content-disposition'].split("=")[1]
            print("Baixando: " + filename)
            with open(filename, 'wb') as file:
                for chunk in request.iter_content(chunk_size=1024): 
                    file.write(chunk)
    return (True,downloadLink);

In [ ]:
def downloadBulk(lat,lon,radius,startDate,endDate,usr,psw,batchSize,productType = "STANDARD", maxDownload = 1, driverPath = '/PATH_REPO/DesmateSeletivo/chromedriver',downloadPath ='/PATH_REPO/DesmateSeletivo'):
    while (apiKey := login(usr,psw)) == None:
           print("Erro ao logar tentando novamente")
    scenes = sceneSearch(apiKey,(lat,lon),radius,startDate,endDate)
    scenesJsonDict = getScenesDict()
    downloadList = []
    pointStr = f'({str(lat)},{str(lon)})'
    for foundScene in scenes['data']['results']:
        if foundScene['displayId'] in scenesJsonDict.keys():
            print("A imagem " + foundScene['displayId'] + " já foi baixada!")
            if pointStr not in scenesJsonDict[foundScene['displayId']]['points']:
                print("Registrando o novo ponto: " + pointStr)
                scenesJsonDict[foundScene['displayId']]['points'].append(pointStr)
        else:
            downloadList.append((foundScene['displayId'],foundScene['downloadUrl'],foundScene['entityId'],foundScene['startTime'],foundScene['endTime']))
    print("Total de imagens para baixar:", len(downloadList))
    for download in downloadList:
            print(download[downloadIndexes['displayId']])
            print("Adicionando para  bulk order:")
            updateBulk(apiKey,'["' +productTypes['full'] + '","' + productTypes['preview'] + '"]',download[downloadIndexes['entityId']])
            print("Obtendo metadados:")
            metaData = metadata(apiKey,'["' + download[downloadIndexes['entityId']] + '"]')
            #print("metadata result: ",metaData)
            cloudCover = None
            for metaDataField in metaData['data'][0]['metadataFields']:
                if metaDataField['fieldName'] == 'Cloud Cover':
                    cloudCover = metaDataField['value']
                    break
            scenesJsonDict[download[downloadIndexes['displayId']]] = {"points" : [pointStr],
                                           "startTime" : download[downloadIndexes['startTime']],
                                           "endTime" : download[downloadIndexes['endTime']],
                                           "cloudCover" : cloudCover,
                                           "sceneBounds" : metaData['data'][0]['sceneBounds'],
                                           "spatialFootprint" : metaData['data'][0]['spatialFootprint'], 
                                           "entityId" : download[downloadIndexes['entityId']],                         
                                           "downloadUrl" : download[downloadIndexes['downloadUrl']],
                                          }
            batchSize += 1
    with open('scenes.json','w+') as jsonFile:
        json.dump(scenesJsonDict,jsonFile)
    logout(apiKey)
    return batchSize

In [ ]:
downloadBulk(-10.463815 ,-58.537751, 500,'2016-01-01','2017-01-01',username,senha, productTypes['preview'])

In [ ]:
downloadBulk(-10.463815 ,-58.537751, 500,'2016-01-01','2016-05-01',username,senha, productType=productTypes['preview'])

In [ ]:
downloadBulk(-10.463815 ,-58.537751, 500,'2016-01-01','2016-01-07',username,senha, productTypes['preview'])

In [ ]:
downloadBulk(-10.463815 ,-58.537751, 500,'2017-07-01','2017-09-30',username,senha, productTypes['preview'])

In [ ]:
def getSceneSearchDisplayIds(apiKey,point,radius,startDate,endDate,
                dataSetName = "SENTINEL_2A", months = None,
                includeUnknownCloudCover = True,maxResults = 1000, sortOrder = 'ASC'):
    ll, ur = calculateMBR(point,radius)
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + dataSetName + '","spatialFilter":{"filterType":"mbr","lowerLeft":{"latitude":"' + str(ll[0]) + '","longitude":"' + str(ll[1]) + '"},"upperRight":{"latitude":"' + str(ur[0]) + '","longitude":"' + str(ur[1]) + '"}},"temporalFilter":{"startDate":"' + startDate + '","endDate":"' + endDate + '"},'+( '' if months is None else '"months":' + str(months) + ',') + '"includeUnknownCloudCover":' + str(includeUnknownCloudCover).lower() + ',"maxResults":"' + str(maxResults) + '","sortOrder":"' + sortOrder + '"}'}
    response = requests.post(defaultUrl + operations['search'], data = payload)
    displayIdList = []
    for scene in response.json()['data']['results']:
        displayIdList.append(scene['displayId'])
    return displayIdList

In [ ]:
with open('Coordenadas desmatamento 2017.csv', mode='r') as original, open('CoordeanadasComNome2017.csv', mode='w') as new:
    reader = csv.reader(original)
    apiKey = login(username,senha)
    writer = csv.writer(new)
    header = next(reader)
    header[1] = "Longitude"
    header[2] = "Latitude"
    print(header,type(header))
    header.append('displayId')
    header.append('displayIdList')
    writer.writerow(header)
    rowN = 0;
    for row in reader:
        listDisplayId = getSceneSearchDisplayIds(apiKey,(float(row[2]),float(row[1])),60,'2017-07-01','2017-09-30')
        row.append(listDisplayId[-1])
        row.append(listDisplayId)
        writer.writerow(row)
        rowN += 1
        print(rowN)
        if rowN > 1000:
            apiKey = login(username,senha)
            rowN = 0

In [ ]:
with open('Coordenadas desmatamento 2017.csv', mode='r') as original, open('CoordeanadasComNome2017.csv', mode='a', newline='') as new:
    reader = csv.reader(original)
    apiKey = login(username,senha)
    writer = csv.writer(new)
    rowN = 0;
    for i in range(10406):
        next(reader)
    for row in reader:
        listDisplayId = getSceneSearchDisplayIds(apiKey,(float(row[2]),float(row[1])),60,'2017-07-01','2017-09-30')
        row.append(listDisplayId[-1])
        row.append(listDisplayId)
        writer.writerow(row)
        rowN += 1
        print(rowN)
        if rowN > 1000:
            apiKey = login(username,senha)
            rowN = 0       

In [ ]:
apiKey = 'be73bcb7be344d5face235f45e14d0c1'
lat = -9.504953
lon =
getSceneSearchDisplayIds(apiKey,(float(row[2]),float(row[1])),60,'2017-07-01','2017-09-30')

In [ ]:
with open('Coordenadas desmatamento 2017.csv', mode='r') as original, open('CoordeanadasComNome2017.csv', mode='a', newline='') as new:
    reader = csv.reader(original)
    apiKey = login(username,senha)
    writer = csv.writer(new)
    rowN = 0;
    for i in range(10406):
        next(reader)
    for row in reader:
        print(row)     

In [ ]:
imgList = []
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        imgList.append(row[4])
c = Counter(imgList)
print(len(c))
print(c)

In [ ]:
imgList = []
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        imgList.append(row[5])
c = Counter(imgList)
print(len(c))
print(c)

In [ ]:
imgNameList = ['L1C_T21LTK_A010732_20170712T142431', 'L1C_T20LRQ_A010732_20170712T142431', 'L1C_T21LTK_A010832_20170719T141344', 'L1C_T21LTK_A011118_20170808T141047', 'L1C_T21LTK_A011304_20170821T142038', 'L1C_T20LRQ_A011304_20170821T142038', 'L1C_T21LTK_A011590_20170910T142035', 'L1C_T20LRQ_A011590_20170910T142035', 'L1C_T21LTK_A011690_20170917T141042', 'L1C_T20LRQ_A011876_20170930T142034', 'L1C_T21LTK_A011876_20170930T142034']
pointsByImage = dict()
for img in imgNameList:
    pointsByImage[img] = []
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        rowList = eval(row[5])
        for imgName in rowList:
            if imgName in imgNameList:
                pointsByImage[imgName].append((row[1],row[2]))
#print(len(pointsByImage))
#print(pointsByImage)
#pointsByImage
for img in pointsByImage:
    print(len(img))

In [ ]:
print(len(pointsByImage['L1C_T21LTK_A010732_20170712T142431']))
print(pointsByImage['L1C_T21LTK_A010732_20170712T142431'])

In [ ]:
points = set()
for i in pointsByImage.values():
    for p in i:
        points.add(p)

In [ ]:
len(points)

In [ ]:
imgFullList = set()
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        rowList = eval(row[5])
        for img in rowList:
            imgFullList.add(img)
print(len(imgFullList))

In [ ]:
line = 0
mean = 0
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        rowList = eval(row[5])
        mean += len(rowList)
        line +=1
print(mean/line)

In [ ]:
def idLookup(apiKey,idList,inputField = 'displayId',datasetName = "SENTINEL_2A"):
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + datasetName + '","idList":"' + idList + '","inputField":"' + inputField + '"}'}
    return requests.post(defaultUrl + operations['idlookup'], data = payload).json()['data'][idList]
        

In [ ]:
def getEntityId(displayId,login,usr,datasetName = ):
    payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + datasetName + '","entityIds":"' + downloadInfo[downloadIndexes['entityId']] + '","products":"' + productType + '"}'}

In [ ]:
def downloadByName(nameList,usr,pwd,productType = "STANDARD",datasetName = "SENTINEL_2A"):
    for displayId in nameList:
        apiKey = login(usr,pwd)
        entityId = idLookup(apiKey,displayId)
        payload = {"jsonRequest" : '{"apiKey":"' + apiKey + '","datasetName":"' + datasetName + '","entityIds":"' + entityId + '","products":"' + productType + '"}'}
        with requests.post(defaultUrl + operations['download'], data=payload) as download:
            downloadLink = (download.json())['data'][0]['url']
            with requests.get(downloadLink, stream=True, allow_redirects=True) as request:
                #lança exceção caso request de errado
                request.raise_for_status()
                filename = request.headers['content-disposition'].split("=")[1]
                print("Baixando: " + filename)
                with open(filename, 'wb') as file:
                    for chunk in request.iter_content(chunk_size=1024): 
                        file.write(chunk)

In [ ]:
idLookup('bfe578d3be5140579115d00cbdf8d400','L1C_T21LXC_A001666_20170701T140052')

In [ ]:
downloadByName(['L1C_T21LXC_A001666_20170701T140052'],username,senha)

In [ ]:
orderSize = 20
actualSize = 0
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    next(reader)
    next(reader)
    for row in reader:
        print("Linnha " + row[0])
        if (actualSize := downloadBulk(float(row[2]),float(row[1]), 200,'2017-07-01','2017-09-30',username,senha,actualSize)) > orderSize:
                break
apiKey = login(username,senha)
submitBulk(apiKey)
logout(apiKey)

In [ ]:
orderSize = 10
actualSize = 0
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    next(reader)
    next(reader)
    for row in reader:
        print("Linnha " + row[0])
        if (actualSize := downloadBulk(float(row[2]),float(row[1]), 200,'2017-07-01','2017-09-30',username,senha,actualSize)) > orderSize:
                break
apiKey = login(username,senha)
submitBulk(apiKey)
logout(apiKey)

In [ ]:
orderSize = 15
actualSize = 0
lines = 200
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for i in range(8936):
        next(reader)
    for row in reader:
        print("Linnha " + row[0])
        actualSize = downloadBulk(float(row[2]),float(row[1]), 200,'2017-07-01','2017-09-30',username,senha,actualSize)
        lines -= 1
        if lines == 0:
            break
if actualSize > 0:
    apiKey = login(username,senha)
    submitBulk(apiKey)
    logout(apiKey)

In [ ]:
apiKey = login(username,senha)
submitBulk(apiKey)
logout(apiKey)

In [ ]:
actualSize

In [ ]:
orderSize = 20
actualSize = 0
with open('CoordeanadasComNome2017-final.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for i in range(736):
        next(reader)
    for row in reader:
        print("Linnha " + row[0])

In [ ]:
with open('/run/media/SEAGATE 1TB/ImagensSentinel2/md5sumNovo.txt') as o, \
     open('/run/media/SEAGATE 1TB/ImagensSentinel2/md5sumNovoHd.txt') as h :
    i = 0
    for line in o:
        i +=1
        fName = line.split('  ')[1][-39:]
        #print('XXXX' + fName)
        for lineH in h:
            fNameH = lineH.split('  ')[1][-39:]
            #print(fName + '==' + fNameH)
            if fName == fNameH:
                #print(line.split('  ')[0] + '==' + lineH.split('  ')[0])
                if line.split('  ')[0] != lineH.split('  ')[0]:
                    print(fName)
                    print(line + "!=" + lineH)
        h.seek(0)
    print(i)

In [2]:
import json

In [3]:
with open('scenes.json','r') as jsonFile:
    j =json.load(jsonFile)
    print(j["L1C_T21LXC_A001666_20170701T140052"])

{'points': ['(-15.904708,-56.000072)', '(-15.416766,-55.988959)', '(-15.437137,-55.914913)', '(-15.468088,-55.897105)', '(-15.535255,-55.968867)', '(-15.564023,-55.94867)', '(-15.568054,-55.882489)', '(-15.588563,-55.870227)', '(-15.598453,-55.881318)', '(-15.611397,-55.846842)', '(-15.605094,-55.847479)', '(-15.548029,-55.817518)', '(-15.555438,-55.815007)', '(-15.546446,-55.818965)', '(-15.581409,-55.697462)', '(-15.574826,-55.668933)', '(-15.56331,-55.668027)', '(-15.566888,-55.664664)', '(-15.613048,-55.53715)', '(-15.613307,-55.549523)', '(-15.597531,-55.56656)', '(-15.59311,-55.563956)', '(-15.615546,-55.565231)', '(-15.667473,-55.595501)', '(-15.58971,-55.802375)', '(-15.597522,-55.802654)', '(-15.598543,-55.811242)', '(-15.617152,-55.819377)', '(-15.619147,-55.820295)', '(-15.643742,-55.825754)', '(-15.773021,-55.85652)', '(-15.784799,-55.944215)', '(-15.778259,-55.750852)', '(-15.806479,-55.653043)', '(-15.891014,-55.602907)', '(-15.896569,-55.587782)', '(-15.90922,-55.598652)

In [5]:
import rasterio
#data link: https://earthexplorer.usgs.gov/download/external/options/SENTINEL_2A/3022286/INVSVC/
pathToImgFolder = "D:\RSDD\ImagensSentinel2\data\\"
pathData = pathToImgFolder + "L1C_T21LXC_A001666_20170701T140052\S2B_MSIL1C_20170701T140049_N0205_R067_T21LXC_20170701T140052.SAFE\GRANULE\L1C_T21LXC_A001666_20170701T140052\IMG_DATA\T21LXC_20170701T140049_B01.jp2" 
boudingBoxCoordinates = (606859.0750363453, 8241169.269917269, 607219.0750363453, 8241529.269917269)
points = [(607014.0750363453,8241374.26991727),(607024.0750363444,8241374.26991727),\
          (607034.0750363453,8241374.26991727),(607044.0750363453,8241374.26991727),\
          (607054.0750363442,8241374.26991727)]
with rasterio.open(pathData) as img:
    bandWindow = rasterio.windows.from_bounds(*boudingBoxCoordinates, img.transform)
    winTransform = rasterio.windows.transform(bandWindow,img.transform)
    bandData = img.read(1, window = bandWindow)
    allData = img.read(1)
    for point in points:
        rBand,cBand = rasterio.transform.rowcol(winTransform,point[0],point[1])
        rFull,cFull = img.index(point[0],point[1])
        bandVal = bandData[rBand,cBand]
        fullVal = allData[rFull,cFull]
        print(f"{fullVal} {'=' if bandVal == fullVal else '!='} {bandVal}")

1202 = 1202
1330 != 1202
1330 != 1202
1330 = 1330
1330 = 1330
